# This notebook is aimed at download HSC cutout images, with a galaxy catalog provided.

In [1]:
import os
import urllib
import glob
import subprocess

import numpy as np

from ipywidgets import widgets

import astropy.wcs as wcs
from astropy.nddata import Cutout2D
from astropy.io import fits
from astropy.table import Table, Column

In [2]:
path = '/Users/jiaxuanli/Research/HSC_Dragonfly_DECaLS/IntermediateZ/'
os.chdir(path)
if not os.path.isdir('Cutout'):
    os.mkdir('Cutout')

## Import catalog

In [3]:
inter_z = Table.read('s16a_massive_z_0.5_logm_11.4_dec_30_for_john.fits', format='fits')
z_0203 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.3, inter_z['z_best']>0.0])]
z_0304 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.4, inter_z['z_best']>0.3])]
z_0405 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.5, inter_z['z_best']>0.4])]

## Download cutout for z_0203 subset

In [4]:
pixel_scale = 0.168 # arcsec / pixel

In [5]:
def gen_url(ra, dec, w, h, band):
    return ['https://hscdata.mtk.nao.ac.jp/das_quarry/dr2.1/cgi-bin/cutout?ra='
       + str(ra) 
       + '&dec='
       + str(dec)
       + '&sw='
       + str(w)
       + 'asec&sh='
       + str(h)
       + 'asec&type=coadd&image=on&variance=on&filter=HSC-'
       + str(band.upper())
       + '&tract=&rerun=s18a_wide']

In [6]:
url_set = []
for obj in z_0203:
    ra, dec = obj['ra'], obj['dec']
    w, h = 100, 100
    band = 'r'
    url_set.append(gen_url(ra, dec, w, h, band))
url_set = np.array(url_set).flatten().tolist()
z_0203.add_column(Column(name='url', data=url_set))

In [7]:
# Write the urls to a file
fh = open('filename.txt', 'w')
for url in url_set[:3]:
    fh.write(url)
    fh.write('\n')
fh.close()

### Download using urllib

In [8]:
# Import HSC username and password
config = Table.read('/Users/jiaxuanli/Documents/HSC_config', format='ascii.no_header')['col1']
username = config[0]
password = config[1]

In [9]:
# Create a password manager
password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()

# Add the username and password.
# If we knew the realm, we could use it instead of None.
top_level_url = 'https://hscdata.mtk.nao.ac.jp/das_quarry/dr2.1/'

password_mgr.add_password(None, top_level_url, username, password)

handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

# create "opener" (OpenerDirector instance)
opener = urllib.request.build_opener(handler)

# use the opener to fetch a URL
opener.open(top_level_url)

# Install the opener.
# Now all calls to urllib.request.urlopen use our opener.
urllib.request.install_opener(opener)

In [10]:
%%time
os.chdir('Cutout/')
if not os.path.isdir('z_0203'):
    os.mkdir('z_0203')
    
for obj in z_0203:
    urllib.request.urlretrieve(
        obj['url'],
        filename=path + 'Cutout/z_0203/' + 'HSC-cutout-ra=' + str(round(obj['ra'], 2)) +
        '&dec=' + str(round(obj['dec'])) + '&band=HSC-r.fits')

CPU times: user 1min 19s, sys: 33.4 s, total: 1min 52s
Wall time: 1h 7min 40s


## In the downloaded `fits` files, layer[1] is image, layer[2] is mask (now empty), and layer[3] is variance.